In [ ]:
#STEP 1: Seeing if I can open this bad boy, simple test performed below with the all data dataFrame object, NYAllData
import pandas as pd

NYAllData = pd.read_csv('New_York_State_Statewide_COVID-19_Testing_20240306.csv')
NYAllData.loc[NYAllData['Geography Description'] == 'St. Lawrence', ['Geography Description']] = 'Saint Lawrence'

NYAllData

In [ ]:
#Num rows = 72
#Getting all of the county names
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.pyplot as plt
import matplotlib.dates
from datetime import datetime

counties=[]
for i in range(0, 72):
    counties.append(NYAllData.at[i, 'Geography Description'])



In [140]:
import geopandas as gpd

map_df = gpd.read_file("Counties_Shoreline.shp")

map_df

,NAME,ABBREV,GNIS_ID,FIPS_CODE,SWIS,NYSP_ZONE,POP1990,POP2000,POP2010,POP2020,DOS_LL,DOSLL_DATE,NYC,CALC_SQ_MI,DATEMOD,Shape_Leng,Shape_Area,geometry
0,Albany,ALBA,974099,36001,010000,East,292594,294565,304204,314848,None,None,N,532.791779,2017-11-10,166077.834242,1.379924e+09,"POLYGON ((608219.061 4737612.320, 608193.470 4..."
1,Allegany,ALLE,974100,36003,020000,West,50470,49927,48946,46456,None,None,N,1035.209131,2019-04-26,210499.339692,2.681179e+09,"POLYGON ((250193.018 4712087.456, 250192.562 4..."
2,Bronx,BRON,974101,36005,600000,Long Island,1203789,1332650,1385108,1472654,None,None,Y,41.933548,2019-10-04,119895.489498,1.086074e+08,"MULTIPOLYGON (((593044.679 4516671.698, 593025..."
3,Broome,BROO,974102,36007,030000,Central,212160,200536,200600,198683,None,None,N,715.287465,2019-04-26,227933.332021,1.852586e+09,"POLYGON ((465447.405 4671514.322, 465436.926 4..."
4,Cattaraugus,CATT,974103,36009,040000,West,84234,83955,80317,77042,None,None,N,1324.309219,2019-04-26,276084.505177,3.429945e+09,"POLYGON ((215534.406 4715146.842, 215536.294 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Washington,WASH,974155,36115,530000,East,59330,61042,63216,61302,None,None,N,840.930564,2017-11-10,342601.088534,2.178000e+09,"MULTIPOLYGON (((629261.070 4824361.974, 629258..."
58,Wayne,WAYN,974156,36117,540000,Central,89123,93765,93772,91283,None,None,N,612.002795,2018-02-08,181763.235088,1.585080e+09,"POLYGON ((360415.647 4800420.303, 360417.878 4..."
59,Westchester,WEST,974157,36119,550000,East,874866,923459,949113,1004457,None,None,N,473.245006,2019-10-04,248650.567653,1.225699e+09,"MULTIPOLYGON (((603994.343 4525951.521, 604008..."
60,Wyoming,WYOM,974158,36121,560000,West,42507,43424,42155,40531,None,None,N,596.822633,2018-09-20,176096.925232,1.545764e+09,"POLYGON ((217004.051 4751905.154, 217012.462 4..."


In [ ]:
#iterating through each county to create a regression model and then plotting it 
NYAllData['Test Date +'] = pd.to_datetime(NYAllData['Test Date'])

for county in counties:
    SpecificCountyData = NYAllData.loc[NYAllData['Geography Description'] == county] 
    #Step 4: Building a model
    
    #doing something I read online to make this date time data actually usable
    #What is basically has done is make Test Date now be the number of days since the start going from 0 (days 1) to the end (the final recored day)
    #Very cheeky
    SpecificCountyData['Test Date +'] = np.arange(len(SpecificCountyData))


    #Adding a cosine feature
    SpecificCountyData['Cos'] = np.cos(2 * np.pi* SpecificCountyData['Test Date +'] / 365)

    X= SpecificCountyData[['Test Date +','Cos']]
    y = SpecificCountyData['Total Cases Per 100k']

    model = LinearRegression().fit(X,y)

    #adding the linear regression data for a county to the NYAllData dataFrame
    #Here is what happens, first a conditional runs to see if the matching county is found
    #then, once this occurs, the model adds a column called 'Linear Regression' to the NYAllData dataFrame
    #This column is then filled with the predicted values of the model
    NYAllData.loc[NYAllData['Geography Description'] == county, 'Linear Regression'] = model.predict(X) #CO-Pilot helped me with this line of code, I have no clue what is going on


    metrics.mean_absolute_error(model.predict(X), SpecificCountyData['Total Cases Per 100k'])
    print("MAE is: ", metrics.mean_absolute_error(model.predict(X), SpecificCountyData['Total Cases Per 100k']))


    x_axis= pd.to_datetime(SpecificCountyData['Test Date']) #This needed to be converted to a date time format to work, used CO-Pilot to help me with this
    y = SpecificCountyData['Total Cases Per 100k']


    plt.xlim(datetime(2020,3,1), datetime(2024,3,3))
    plt.ylim(0, 660)
    plt.scatter(x_axis, y, color = 'green')

    plt.plot(x_axis, model.predict(X), color="blue") #CO-Pilot added the model.predict(X) part, and I think I understand what it is doing
    plt.xlabel("Test Date")
    plt.ylabel("Total Cases Per 100k")
    plt.title(county +", MAE: "+ str(metrics.mean_absolute_error(model.predict(X), SpecificCountyData['Total Cases Per 100k'])))
    plt.show()

In [ ]:
NYAllData

In [ ]:
#plotting NY state
map_df.plot()

In [143]:

mask = ((NYAllData['Test Date +'] == "2024-03-03") & (NYAllData['Geography Level'] == "COUNTY"))
OneDayData = NYAllData[mask]
OneDayData

,Test Date,Geography Description,Geography Level,Total New Positives,Total Cases Per 100k,Total Cases Per 100k (7-day avg),New Positives PCR,Cases Per 100k PCR,New Positives Antigen,Cases Per 100k Antigen,Test Date +,Linear Regression
0,03/03/2024,Albany,COUNTY,9,2.85,2.80,8,2.53,1,0.32,2024-03-03,21.280676
1,03/03/2024,Allegany,COUNTY,2,4.28,2.45,1,2.14,1,2.14,2024-03-03,15.499679
2,03/03/2024,Bronx,COUNTY,51,3.70,5.99,26,1.88,25,1.81,2024-03-03,36.217666
3,03/03/2024,Broome,COUNTY,3,1.52,3.19,3,1.52,0,0.00,2024-03-03,28.102199
4,03/03/2024,Cattaraugus,COUNTY,1,1.31,2.43,0,0.00,1,1.31,2024-03-03,17.892996
...,...,...,...,...,...,...,...,...,...,...,...,...
57,03/03/2024,Washington,COUNTY,2,3.29,4.70,1,1.64,1,1.64,2024-03-03,28.378866
58,03/03/2024,Wayne,COUNTY,1,1.10,5.17,1,1.10,0,0.00,2024-03-03,24.023592
59,03/03/2024,Westchester,COUNTY,41,4.14,7.75,22,2.22,19,1.92,2024-03-03,34.166797
60,03/03/2024,Wyoming,COUNTY,1,2.52,4.68,1,2.52,0,0.00,2024-03-03,19.436627


In [144]:
#Joing the data frames 
map_df = map_df.set_index('NAME').join(OneDayData.set_index('Test Date'))

In [145]:
map_df

,ABBREV,GNIS_ID,FIPS_CODE,SWIS,NYSP_ZONE,POP1990,POP2000,POP2010,POP2020,DOS_LL,...,Geography Level,Total New Positives,Total Cases Per 100k,Total Cases Per 100k (7-day avg),New Positives PCR,Cases Per 100k PCR,New Positives Antigen,Cases Per 100k Antigen,Test Date +,Linear Regression
Albany,ALBA,974099,36001,010000,East,292594,294565,304204,314848,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
Allegany,ALLE,974100,36003,020000,West,50470,49927,48946,46456,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
Bronx,BRON,974101,36005,600000,Long Island,1203789,1332650,1385108,1472654,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
Broome,BROO,974102,36007,030000,Central,212160,200536,200600,198683,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
Cattaraugus,CATT,974103,36009,040000,West,84234,83955,80317,77042,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Washington,WASH,974155,36115,530000,East,59330,61042,63216,61302,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
Wayne,WAYN,974156,36117,540000,Central,89123,93765,93772,91283,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
Westchester,WEST,974157,36119,550000,East,874866,923459,949113,1004457,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
Wyoming,WYOM,974158,36121,560000,West,42507,43424,42155,40531,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
